In [1]:
import pandas as pd
import numpy as np

from fuzzywuzzy import process
from fuzzywuzzy import fuzz


In [2]:
pwd

'C:\\Users\\rohan'

In [28]:
# make a list of all duplicate company names in the master by assigning 'keep = False'

duplicate_master = working_master[working_master.duplicated(subset = 'COMPANY NAME', keep=False)].reset_index(drop=True)

# make a list of all unique company names in the master by assigning 'keep = False' and inversing. We cannot use '.unique()' since
# it takes 1 name from duplicate names also.

unique_master = working_master[~working_master.duplicated(subset = 'COMPANY NAME', keep=False)].reset_index(drop=True)

## cleaning the duplicate_master

header_list = working_master.columns.values.tolist()
unique_duplicates = pd.DataFrame(columns = header_list)   ## blank dataframe to keep outputs. Will have same companies with 
                                                          ## different addresses

# grouping all duplicate company names :

grouped_duplicate = duplicate_master.groupby(['COMPANY NAME'])  
duplicate_name_list = list(grouped_duplicate.groups)         ## this gives the names of the unique companies in duplicate list
print('num of duplicate companies=',len(duplicate_name_list)) ## this gives the number of unique comany names in duplicate list

# what to keep in duplicate - companies with same addresses to be discarded :

thres = 80

for i in range(len(duplicate_name_list)):

    
    current_group = grouped_duplicate.get_group(duplicate_name_list[i]).reset_index(drop=True)
    
    current_group = current_group.replace(r'^\s+$', np.nan, regex=True) #all blanks to be converted to Nan
    current_group[['ADD']] = current_group[['ADD']].fillna('nil')   #convert Nan in address column to 'nil'
     
    add_list = current_group['ADD'].tolist()
    
    drop_list_1 = []                                                # to be used to drop duplicate companies whin a group

#   dealing with a single group 'i':

    for j in range(len(current_group)):
        drop_list_2 = []

        if (len(current_group)-len(drop_list_1)) > 1:               # to safeguard if only 1 name remains after dropping

            drop_list_2.append(drop_list_1)
            drop_list_2.append(j)
            comp_list = [k for m, k in enumerate(add_list) if m not in drop_list_2] # remaining addresses after dropping+current
        
#            best_match = process.extractOne(current_group['ADD'][j], comp_list, scorer = fuzz.token_set_ratio)
            best_match = process.extractOne(current_group['ADD'][j], comp_list)
        
            if best_match[1] >= thres:
                add_list[j] = 'x'
                indx = add_list.index(best_match[0])  ## so, 'j' address matches with 'indx' address
                
                if current_group.count(1)[j] >= current_group.count(1)[indx]:  # name row to be dropped where 'NaN' is maxm
                    drop_list_1.append(indx)          

                else:
                    drop_list_1.append(j)
                    
    current_group.drop(current_group.index[drop_list_1], inplace=True)
    unique_duplicates = unique_duplicates.append(current_group, ignore_index=True)

final_master = unique_master.append(unique_duplicates, ignore_index=True)

tz_data_rev.to_excel('tranzact_data.xlsx', index=False, engine='xlsxwriter')
final_master.to_excel('master_co_data.xlsx', index=False, engine='xlsxwriter')


num of duplicate companies= 25175


c:\users\rohan\appdata\local\programs\python\python39\lib\site-packages\xlsxwriter\worksheet.py:1183: UserWarning: Ignoring URL 'https://plus.google.com/104769250087782069184https://plus.google.com/collection/YwBADE%20ABOUT%20SLU%20GROUPWe%20are%20a%20water%20treatment%20company.%20Who%20has%20been%20providing%20professional%20and%20value%20added%20consultancy%20and%20technical%20service%20to%20customers%20in%20Asian%20region%20since%202014.%20With%20the%20long%20experience%20in%20the%20water%20treatment%20we%20combines%20innovative%20technologies%20and%20services%20provided%20by%20our%20high%20skilled%20representatives.Thanks%20to%20an%20intensive%20research%20and%20development%20and%20our%20own%20production%20facilities%20we%20can%20offer%20and%20extensive%20range%20of%20products%20as%20well%20as%20tailor%20made%20solutions%20to%20cover%20every%20specific%20application%20and%20help%20you%20optimize%20the%20efficiency%20of%20your%20water%20processes%20achieving%20the%20greatest%20econ

In [5]:
final_master.head(3)

,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
0,1,paradigm granite pvt. ltd.,Survey No. 293/2/3 & 295/2A/2B/2D/2E/ Nallagan...,Hosur,635117,Tamil Nadu,NaN,91-9880619554,paradigmgranite@yahoo.com,www.paradigmgranite.com,"We, Paradigm granite is a 100% EOU having busi..."
1,2,ramac tools & instruments pvt. ltd.,"# 49, Madhukar Building, 3rd Floor, Cubbon Lan...",Bengaluru,560001,Karnataka,91-80-41132705,91-9845039918,info@ramactools.net,www.ramactools.net,Ramac tools and instruments Pvt. Ltd. Is assoc...
2,3,southern rocks & minerals pvt. ltd.,Adjacent to Industrial Estate Kurnool Road,Ongole,523002,Andra Pradesh,91-8592-232588,91-9849545487,sourox@rediffmail.com,www.southernrocksandminerals.com,Southern Rocks & Minerals (P) LTD has the bato...


In [6]:
tz_data_rev.head(3)

,id,name,mobile_no,about,company_image,no_of_employees,ownership,sector,email_blacklist,buy,...,default,gstin,gstin_type,company_profile,external_profile,address_id,company_id,created_by_id,display_email,last_modified_date
0,90014,gurukrupa,NaN,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,0,0,NaN,85327,92416.0,rewali6372@tlhao86.com,2021-04-03 19:06:28.117
1,90263,gepl,NaN,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,0,0,NaN,85569,92672.0,pradeepkurmi1473@gmail.com,2021-04-06 09:20:25.219
2,90767,skytech engineering,NaN,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaN,0,0,NaN,86041,93184.0,info.skytechengineering@gmail.com,2021-04-11 17:33:00.730


In [2]:
## Load required python modules :

import pandas as pd
import numpy as np

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

## Load the data files :

final_master = pd.read_excel(r'.\master_co_data.xlsx', header = 0)
tz_data_rev = pd.read_excel(r'.\tranzact_data.xlsx', header=0)


In [12]:
## Company matching

company_list = final_master['COMPANY NAME'].tolist()  ## master company list with which to match
add_master_list = final_master['ADD'].tolist()

start_row = 200        ## start company
num_co = 100           ## number of companies to match

thresh_1 = 95          ## matching thresold
thresh_2 = 90          ## matching thresold

tz_data_short = tz_data_rev[start_row:num_co+start_row].reset_index(drop=True)  ## taking limited data as per number of companies to match

final_list = []       ## This the output list
unmatched_list = []   ## unmatched list

for i in range(len(tz_data_short)):
        
    match_1 = []
    match_2 = []
    match_3 = []
    tz_unmatched = []
    list_ratios = []
    
    name_check = tz_data_short.name[i]
    ratios = process.extract(name_check, company_list, limit=15)  ## matching with master file-gives best 15 matches as per limit
#    print(ratios)
    for m in range(len(ratios)):
        list_ratios.append(ratios[m][1])
#    print('*******************************************')
#    print('list_ratios = ', list_ratios)
    
    index_list = []
    add_list = []
    id_list = []
    sim_com = []
    match_score = []
    check_list = company_list.copy()
    
    for j in range(len(ratios)):
        if ratios[j][1] > thresh_1:
            id_x = check_list.index(ratios[j][0])
            index_list.append(id_x)
            match_2.append(ratios[j][1])
            check_list[id_x] = 'x'
            add_list.append(final_master['ADD'][id_x])
                        
        elif thresh_2 < ratios[j][1] <= thresh_1:
            id_x = check_list.index(ratios[j][0])
            id_list.append(id_x)
            match_3.append(ratios[j][1])
            check_list[id_x] = 'x'
            
#    print('match_2 = ', match_2)
#    print('match_3 = ', match_3)
#    print('length_index_list = ', len(index_list))
#    print('length_id_list = ', len(id_list))
#    print('index_list = ', index_list)
    if len(index_list) > 0:
            
        match_1 = final_master.iloc[index_list[0]].tolist()  # best match
        sim_com = 'None'
#        print('best match = ', index_list[0] )
        if len(index_list) > 1:
#=====================================================================================================================

            add_check = final_master['ADD'][index_list[0]]
            add_check_list = add_list[1:len(add_list)]       # since 1st address is the best match hence skip 0th address

            add_m_ratios = process.extract(add_check, add_check_list, limit=14)  ## this is the problem line
            for k in range(len(add_m_ratios)):
                if add_m_ratios[k][1] >= 90:
                    remove_add_index = add_list.index(add_m_ratios[k][0])
                    index_list.pop(remove_add_index)
                    match_2.pop(remove_add_index)

#======================================================================================================================
            sim_com = index_list[1:]
            if len(sim_com) == 0:
                sim_com = 'None'
            match_score = match_2[1:]
                
        match_1.append(ratios[0][1])
        match_1.append(sim_com)
        match_1.append(match_score)
        match_1.append(name_check)
    
        final_list.append(match_1)                 ## Best match
       
    else:
#        print('+++++++++++++++++++++++++++++++')
        tz_unmatched = tz_data_short.iloc[i].tolist()
        
        if len(id_list) == 0:
            tz_unmatched.append('none')
        else:
            tz_unmatched.append(id_list)
            
        unmatched_list.append(tz_unmatched)
             
#    final_list.append(match_2)                 ## 2nd best match
    
## Creating dataframe from output list :
    
heading_list = ['S_No', 'Company_name', 'Add', 'City', 'Pin', 'State', 'Cont_no', 'Mob_no', 'Email', 'Web',
                'Details', 'M_score', 'similar', 'score', 'TZ_name']

company_details = pd.DataFrame(final_list, columns=heading_list)   ## matched result


## Re-organise columns of above company_details:

matched_final = company_details[['TZ_name', 'S_No', 'Company_name', 'M_score', 'similar', 'score', 'Add', 'City', 'Pin', 'State', 'Cont_no',
                              'Mob_no', 'Email', 'Web', 'Details']]

unmatched_header = list(tz_data_short.columns.values)
unmatched_header.append('Can_try')
unmatched_details = pd.DataFrame(unmatched_list, columns=unmatched_header)
unmatched_final = unmatched_details[['id','name','Can_try','mobile_no','about','company_image','no_of_employees','ownership',
                                      'sector','email_blacklist','buy','sell','active','creation_date','default','gstin',
                                      'gstin_type','company_profile','external_profile','address_id','company_id','created_by_id',
                                      'display_email','last_modified_date']]

matched_final.to_excel('matched_final_3.xlsx', index=False, engine='xlsxwriter')
unmatched_final.to_excel('unmatched_final_3.xlsx', index=False, engine='xlsxwriter')

print("Finished processing all files")


Finished processing all files


In [8]:
matched_final.tail()

,TZ_name,S_No,Company_name,M_score,similar,score,Add,City,Pin,State,Cont_no,Mob_no,Email,Web,Details
21,supreme engineers,906268,supreme engineers,100,"[677632, 397470]","[100, 97]",J-3 Sector 18,Noida,201301,NaN,NaN,9810750810,NaN,NaN,Real Estate Agents & Dealers
22,kajay remedies pvt ltd,602360,kajay remedies pvt. ltd.,98,None,[],"3974, Burna Street Sadar Bazar",Delhi,110006,Delhi,NaN,9810081893,arushi_jain2001@yahoo.co.in,NaN,"Potato Pellets, Fryms, Fryums"
23,karan creation,114138,karans creation,97,[532287],[96],"1/3, Sujata Chambers, Base-1, Abhechand Gandhi...",MUMBAI,400009,Maharashtra,912223417187,NaN,mashamshi2006@yahoo.co.in,NaN,"Incense, Agarbattis, DhoopSticks, Dhoop Cones,..."
24,m.s. enterprises,56451,m.s. enterprises,100,"[117933, 656686, 656687, 392990, 656548, 65654...","[100, 100, 100, 97, 97, 97, 97, 97, 97]","2193/3, Chuna Mandi, Pahar Ganj",Delhi,NaN,Delhi,NaN,91-9811345800,NaN,NaN,Food equipments & Beverage Equipments ( Prepar...
25,noveltex pvt. ltd.,619364,noveltex pvt. ltd.,100,[475544],[97],149/150 Jayanagar 10th A Main Road 1st Block,Bangalore,560011,NaN,80-26561168,9449051168,NaN,NaN,Real Estate Consultants


In [9]:
unmatched_final.tail()

,id,name,Can_try,mobile_no,about,company_image,no_of_employees,ownership,sector,email_blacklist,...,default,gstin,gstin_type,company_profile,external_profile,address_id,company_id,created_by_id,display_email,last_modified_date
69,86648,planet multi car service centre,none,9320444551,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,...,1,27AATFP3745Q2Z6,Regular,0,0,88399.0,82166,32012.0,planetmulticar2017@gmail.com,2021-03-04 16:24:34.132
70,89069,samir auto care,none,8446124524,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,...,1,NaN,Unregistered,0,0,90379.0,84453,32012.0,samirautocare@gmail.com,2021-03-24 13:02:51.969
71,87247,auto planet wheels,none,7039103746,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,...,1,27ABBFA0811H1Z0,Regular,0,0,88874.0,82718,32012.0,autoplanetwheels@gmail.com,2021-03-12 11:53:00.754
72,89556,dnyanaee enterprises,[95091],7507818580,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,...,1,NaN,Unregistered,0,0,90728.0,84902,32012.0,amolgore1087@gmail.com,2021-03-30 17:12:27.322
73,86883,varroc engineering limited,[276127],8605156739,NaN,default_images/company_default.jpg,NaN,NaN,NaN,NaN,...,1,27AAACV2420J1ZI,Regular,0,0,88600.0,82386,32012.0,SKAZI@varroclighting.com,2021-03-08 16:29:58.884


In [10]:
final_master[95091:95092]

,S.No,COMPANY NAME,ADD,CITY,PIN,STATE,CONTACT NO.,MOBILE No.,EMAIL,WEB,DETAILS
95091,124961,yanee enterprises,"W-283, MIDC, Rabale,",Navi Mumbai,400701,Maharashtra,22-27690743,NaN,aditi21@yahoo.com,http://www.jpnonwoven.com,"Home Furnishings,Mfrs. & Expt. Of Non-Wovens N..."
